<a href="https://colab.research.google.com/github/ngwalker93/ADS-507-Final-Team-Project/blob/main/ADS507_Final_FDA_Shortage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import pandas as pd
import requests
import zipfile
import io
import os
import json

url = "https://download.open.fda.gov/drug/ndc/drug-ndc-0001-of-0001.json.zip"

# Download the file
response = requests.get(url)
response.raise_for_status()

zip_bytes = io.BytesIO(response.content)

with zipfile.ZipFile(zip_bytes) as z:
    z.extractall("data")

json_path = "data/drug-ndc-0001-of-0001.json"

# Load JSON file first, then extract the results
with open(json_path, 'r') as f:
    data = json.load(f)

# The actual drug data is in the 'results' key
df = pd.DataFrame(data['results'])

df.head()

,product_ndc,generic_name,labeler_name,brand_name,active_ingredients,finished,packaging,listing_expiration_date,openfda,marketing_category,...,product_type,route,marketing_start_date,product_id,application_number,brand_name_base,pharm_class,dea_schedule,brand_name_suffix,marketing_end_date
0,75712-940,"Avobenzone, Homosalate, Octisalate",Old East Main Co.,"Avobenzone, Homosalate, Octisalate","[{'name': 'AVOBENZONE', 'strength': '30 mg/mL'...",True,"[{'package_ndc': '75712-940-34', 'description'...",20261231,"{'manufacturer_name': ['Old East Main Co.'], '...",OTC MONOGRAPH DRUG,...,HUMAN OTC DRUG,[TOPICAL],20230824,75712-940_3864ec25-a45a-15a1-e063-6394a90a56c2,M020,"Avobenzone, Homosalate, Octisalate",NaN,NaN,NaN,NaN
1,75712-944,"Avobenzone, Homosalate, Octisalate",Old East Main Co.,"Avobenzone, Homosalate, Octisalate","[{'name': 'AVOBENZONE', 'strength': '30 mg/g'}...",True,"[{'package_ndc': '75712-944-39', 'description'...",20261231,"{'manufacturer_name': ['Old East Main Co.'], '...",OTC MONOGRAPH DRUG,...,HUMAN OTC DRUG,[TOPICAL],20230808,75712-944_38656bb5-a569-761a-e063-6294a90a77e0,M020,"Avobenzone, Homosalate, Octisalate",NaN,NaN,NaN,NaN
2,75742-003,ALCOHOL,"Zhejiang Guoyao Aerosol Co., Ltd",Green Island Sanitiser,"[{'name': 'ALCOHOL', 'strength': '75 mL/100mL'}]",True,"[{'package_ndc': '75742-003-01', 'description'...",20271231,{'manufacturer_name': ['Zhejiang Guoyao Aeroso...,OTC MONOGRAPH DRUG,...,HUMAN OTC DRUG,[TOPICAL],20200330,75742-003_47af947b-d29d-7000-e063-6394a90ad871,M003,Green Island Sanitiser,NaN,NaN,NaN,NaN
3,75742-007,Ethyl Alcohol,"Zhejiang Guoyao Jingyue Aerosol Co., Ltd.",Healing Solutions,"[{'name': 'ALCOHOL', 'strength': '75 mL/100mL'}]",True,"[{'package_ndc': '75742-007-01', 'description'...",20271231,{'manufacturer_name': ['Zhejiang Guoyao Jingyu...,OTC MONOGRAPH DRUG,...,HUMAN OTC DRUG,[TOPICAL],20200706,75742-007_47afb8dc-35f7-cf1b-e063-6394a90a0371,M003,Healing Solutions,NaN,NaN,NaN,NaN
4,75742-027,Ethyl Alcohol,"Zhejiang Guoyao Jingyue Aerosol Co., Ltd.",Hand sanitizer 2oz 2pk original,"[{'name': 'ALCOHOL', 'strength': '70 mL/100mL'}]",True,"[{'package_ndc': '75742-027-01', 'description'...",20261231,{'manufacturer_name': ['Zhejiang Guoyao Jingyu...,OTC MONOGRAPH DRUG,...,HUMAN OTC DRUG,[TOPICAL],20231004,75742-027_06dd2427-097d-b0fa-e063-6294a90a4f82,M003,Hand sanitizer 2oz 2pk original,NaN,NaN,NaN,NaN


In [20]:
# List all column names in the raw NDC dataset

df.columns.tolist()

['product_ndc',
 'generic_name',
 'labeler_name',
 'brand_name',
 'active_ingredients',
 'finished',
 'packaging',
 'listing_expiration_date',
 'openfda',
 'marketing_category',
 'dosage_form',
 'spl_id',
 'product_type',
 'route',
 'marketing_start_date',
 'product_id',
 'application_number',
 'brand_name_base',
 'pharm_class',
 'dea_schedule',
 'brand_name_suffix',
 'marketing_end_date']

In [21]:
# Create a clean, flat core NDC products table to serve as the primary
# reference table for joining with FDA drug shortage data.
# This table keeps only one-to-one product attributes (no nested fields).

ndc_core_cols = [
    "product_ndc",              # Primary identifier / join key
    "brand_name",
    "generic_name",
    "labeler_name",
    "product_type",
    "marketing_category",
    "dosage_form",
    "route",
    "finished",
    "listing_expiration_date",
    "marketing_start_date",
    "marketing_end_date",
    "application_number",
    "pharm_class",
    "dea_schedule"
]

ndc_products = df[ndc_core_cols].copy()

ndc_products.head()


,product_ndc,brand_name,generic_name,labeler_name,product_type,marketing_category,dosage_form,route,finished,listing_expiration_date,marketing_start_date,marketing_end_date,application_number,pharm_class,dea_schedule
0,75712-940,"Avobenzone, Homosalate, Octisalate","Avobenzone, Homosalate, Octisalate",Old East Main Co.,HUMAN OTC DRUG,OTC MONOGRAPH DRUG,LOTION,[TOPICAL],True,20261231,20230824,NaN,M020,NaN,NaN
1,75712-944,"Avobenzone, Homosalate, Octisalate","Avobenzone, Homosalate, Octisalate",Old East Main Co.,HUMAN OTC DRUG,OTC MONOGRAPH DRUG,SPRAY,[TOPICAL],True,20261231,20230808,NaN,M020,NaN,NaN
2,75742-003,Green Island Sanitiser,ALCOHOL,"Zhejiang Guoyao Aerosol Co., Ltd",HUMAN OTC DRUG,OTC MONOGRAPH DRUG,SPRAY,[TOPICAL],True,20271231,20200330,NaN,M003,NaN,NaN
3,75742-007,Healing Solutions,Ethyl Alcohol,"Zhejiang Guoyao Jingyue Aerosol Co., Ltd.",HUMAN OTC DRUG,OTC MONOGRAPH DRUG,GEL,[TOPICAL],True,20271231,20200706,NaN,M003,NaN,NaN
4,75742-027,Hand sanitizer 2oz 2pk original,Ethyl Alcohol,"Zhejiang Guoyao Jingyue Aerosol Co., Ltd.",HUMAN OTC DRUG,OTC MONOGRAPH DRUG,GEL,[TOPICAL],True,20261231,20231004,NaN,M003,NaN,NaN


In [22]:
# Confirm how many rows and columns are in the core NDC table

ndc_products.shape



(130883, 15)

In [23]:
# Show the proportion of missing values for each column in the core NDC table

ndc_products.isna().mean().sort_values(ascending=False)


,0
marketing_end_date,0.965053
dea_schedule,0.952171
pharm_class,0.457477
application_number,0.282122
route,0.175286
brand_name,0.162023
listing_expiration_date,0.034963
generic_name,0.000023
dosage_form,0.000000
product_type,0.000000


In [24]:
# Create a one-to-many packaging table linked by product_ndc

packaging_rows = []

for _, row in df[["product_ndc", "packaging"]].dropna().iterrows():
    for pkg in row["packaging"]:
        pkg_row = {
            "product_ndc": row["product_ndc"],
            "package_ndc": pkg.get("package_ndc"),
            "description": pkg.get("description"),
            "marketing_start_date": pkg.get("marketing_start_date"),
            "marketing_end_date": pkg.get("marketing_end_date")
        }
        packaging_rows.append(pkg_row)

ndc_packaging = pd.DataFrame(packaging_rows)

ndc_packaging.head()

,product_ndc,package_ndc,description,marketing_start_date,marketing_end_date
0,75712-940,75712-940-34,"236 mL in 1 BOTTLE, PLASTIC (75712-940-34)",20230824,None
1,75712-944,75712-944-39,312 g in 1 CAN (75712-944-39),20230808,None
2,75742-003,75742-003-01,"30 mL in 1 BOTTLE, SPRAY (75742-003-01)",20200330,None
3,75742-003,75742-003-02,"60 mL in 1 BOTTLE, SPRAY (75742-003-02)",20200330,None
4,75742-003,75742-003-03,"80 mL in 1 BOTTLE, SPRAY (75742-003-03)",20200330,None


In [25]:
# Confirm multiple packaging records exist per product_ndc

ndc_packaging.groupby("product_ndc").size().sort_values(ascending=False).head()

,0
product_ndc,
59579-002,63
84165-016,59
84165-102,58
84165-006,55
84165-002,54


In [26]:
# Save reduced tables locally for reuse in later steps

ndc_products.to_csv("data/ndc_products.csv", index=False)
ndc_packaging.to_csv("data/ndc_packaging.csv", index=False)